In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import re
import os
from pathlib import Path
import gc
from zipfile import ZipFile
from unidecode import unidecode

project_dir = Path("notebook.ipynb").resolve().parents[0]

In [2]:
#criando dataframe de tarifas de energia

tarifa_energia = pd.read_csv(f'{project_dir}/data/TarifaMediaFornecimento.csv', encoding='latin-1')
values_to_drop = tarifa_energia[tarifa_energia['Unnamed: 7'].notnull()].index.tolist()
tarifa_energia = tarifa_energia.drop(values_to_drop,axis=0)
tarifa_energia = tarifa_energia.drop('Unnamed: 7', axis=1)
tarifa_energia = tarifa_energia.rename(columns={'anoReferencia':'ano','nomClasseConsumo':'tipo_consumo','vlrConsumoMWh':'valor_mwh_tarifa'})
tarifa_energia = tarifa_energia[tarifa_energia.nomRegiao=='Sul']
tarifa_energia['ano'] = tarifa_energia['ano'].astype('str')
tarifa_energia = tarifa_energia[tarifa_energia.valor_mwh_tarifa.str.len()==6]
tarifa_energia = tarifa_energia.astype({'ano':'str'})

#dataframe com valores de ipca
ipca = pd.read_csv(f'{project_dir}/data/ipca_historico.csv', sep=';',skiprows=1)
ipca.columns = ipca.columns=['periodo','ipca_value']
ipca = ipca.drop(26, axis=0)

def return_year(x):
    return re.sub(r'\D*','',x)


ipca['ano'] = ipca['periodo'].map(return_year)
ipca = ipca[['ano','ipca_value']]
ipca = ipca.astype({'ano':'str','ipca_value':'float'})


#dataframe para tarifas TUSD e TE da CEEE
tusd_ceee=[522.3,518.1,400.5,298.0,328.0,417.0,302.6,389.9,350.9,365.4,345.1,345.1]
te_ceee =[269.0,273.5,325.1,346.7,433.9,471.3,355.9,456.0,480.6,392.8,323.2,323.2]
anos=[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]

energia_ceee = pd.DataFrame(columns=anos, data=[tusd_ceee, te_ceee])
energia_ceee['categoria_tarifa'] = ['TUSD CEEE','TE CEEE']
energia_ceee = energia_ceee.melt(id_vars=['categoria_tarifa'], var_name='ano',value_name='tarifa_mwh')
energia_cee = energia_ceee.astype({'ano':'str', 'tarifa_mwh':'float'})

In [4]:
# lẽ todos os arquivos .zip e descompacta na pasta raw_data:
for archive in os.listdir(f'{project_dir}/raw_data'):
    with ZipFile(f'{project_dir}/raw_data/{archive}','r') as zipfile:
        zipfile.extractall(f'{project_dir}/data/')
total=0

gc.collect()

#transforma os dados da forma wide para forma long, além de padronizar nomes de colunas:
for file in os.listdir(f'{project_dir}/data/'):
    if file == 'TarifaMediaFornecimento.csv':
        continue
    if file =='ipca_historico.csv':
        continue
    else:
        data = pd.read_csv(f'{project_dir}/data/{file}',sep='\n', encoding='latin-1', header=None)
        data = data[0].str.split(';', expand=True)
        data = data.reset_index(drop=True)
        var_name = data[0][1][7:]
        var_name = unidecode(var_name).lower().strip().replace(' ','_').replace('/','_').replace(':','')
        data.columns = data.iloc[5].tolist()
        data = data.drop([0,1,2,3,4,5], axis=0).reset_index(drop=True)
        data = data.drop(columns=['latitude','longitude'])
        for column in data.columns[2:]:
            column_name = re.sub(r'\D*','',column)
            data = data.rename(columns={column:f'{column_name}'})
        
        data_melted = data.melt(id_vars=['Município','ibge'], var_name=f'ano', value_name=f'{var_name}_value')
        data_melted.to_csv(f'{project_dir}/melted_data/{var_name}_value.csv', sep=';', decimal=',', encoding='latin-1', index=False)
    total+= 1

melted_data = pd.read_csv(f'{project_dir}/melted_data/__educacao_ensino_fundamental_taxa_de_abandono_total_value.csv', header=None, sep='\n',encoding='latin-1')
melted_data = melted_data[0].str.split(';', expand=True)
melted_data.columns= melted_data.iloc[0]
melted_data = melted_data.drop([0], axis=0)
melted_data = melted_data.astype({'ibge':'str','ano':'str'})

gc.collect()


#concatena todos os arquivos em um único dataset:
for file in os.listdir(f'{project_dir}/melted_data/'):
    if file =='__educacao_ensino_fundamental_taxa_de_abandono_total_value.csv':
        continue
    df= pd.read_csv(f'{project_dir}/melted_data/{file}', header=None, sep='\n', encoding='latin-1')
    df = df[0].str.split(';', expand=True)
    df.columns = df.iloc[0]
    df = df.drop([0], axis=0)
    df = df.astype({'ibge':'str','ano':'str'})
    df = df.sort_values(['Município','ano'])
    melted_data = melted_data.merge(df, on=['Município','ibge','ano'], suffixes=('','_D'), how='outer')
    melted_data = melted_data.drop(melted_data.filter(regex='_D$').columns.tolist(), axis=1)
    #melted_data.to_csv(f'{project_dir}/preliminar_dataset.csv', sep=';',decimal=',', encoding='latin-1')


def cut(x):
    return str(x)[:4]


final_dataset = melted_data
final_dataset = final_dataset.astype({'ano':'str'})
final_dataset['ano'] = final_dataset['ano'].map(cut)
final_dataset = final_dataset.merge(energia_ceee, on='ano', how='outer')
final_dataset = final_dataset.merge(ipca, on='ano', how='outer')
final_dataset.to_csv(f'{project_dir}/final_dataset.csv', sep=';',decimal=',', encoding='latin-1', index=None)
print(f'Tamanho final: {final_dataset.shape}')

Tamanho final: (27966, 176)
